In [52]:
import os
from dotenv import load_dotenv
import re

from groq import Groq

## Get the API key

In [53]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
print(len(GROQ_API_KEY))

56


## Client

In [54]:
client = Groq(
    api_key=GROQ_API_KEY,
)

## Prompts

In [55]:
# CLASS_PROMPT : classifier prompt
# NEWS_PROMPT : current events prompt
# COMPANY_PROMPT : company prompt
# THINK_PROMPT : thinking companion prompt

In [56]:
CLASS_PROMPT = """
Look at the user query.
Is it a question about a current event / political topic? If so, return 1.
Is it a question about a company or an investment opportunity? If so, return 2.
Is it a general question requiring a thinking companion? If so, return 3.
If it's impossible to fit any of the preceding descriptions, return 4.
Don't return anything else. Just one number. Nothing else. No explanations. You should return one single symbol. Double-check that.
"""

In [57]:
NEWS_PROMPT = """
You are a talented political commentator who's able to comment on a political event from multiple angles. 
You will be given a current political event/topic. 
Return a JSON object with the following data (look at the name of the parameter and its explanation): 
    "topic", // The name of the topic/event
    "timeline", // Date, like January 2020 -- March 2022
    "summary",  // This is a compressed UNBIASED summary of the commentaries from different political views
    "left_wing_commentary", // What is the left-wing view of the event?
    "right_wing_commentary", // What is the right-wing view of the event?
    "list_of_last_news". // Every piece of news is structured as follows: "news_story", "date", "source", "bias_level". "bias_level" is a float between 0.0 (unbiased) and 1.0 (extremely biased and one-sided).
Make sure that you respect the provided schema of the JSON object.
Don't return anything else. Just the JSON.
    """

## Classifier

In [58]:
def get_mode(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": CLASS_PROMPT,
            },
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    model_output = chat_completion.choices[0].message.content

    # Make sure that the classifier outputs a single number
    regex = re.compile(r'(\d+).*', flags=re.DOTALL)
    result = regex.search(model_output).group(1)
    result = int(result) if result is not None else 4

    return result

In [59]:
print(get_mode("Ukraine War"))
print(get_mode("Tesla, Inc."))
print(get_mode("Let's discuss Daniel Kahneman's book, Thinking Fast and Slow."))
print(get_mode("whatever whatever whatever whatever"))

1
2
3
4


## Quering

In [40]:
def news_query(user_input):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": NEWS_PROMPT,
            },
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model="mixtral-8x7b-32768",
    )

    return chat_completion.choices[0].message.content

In [42]:
print(news_query("Ukraine war"))

{
  "topic": "Ukraine War",
  "timeline": "February 2014 -- Present",
  "summary": "The Ukraine war is a complex and multifaceted conflict that began in 2014, following the Ukrainian Revolution and the annexation of Crimea by the Russian Federation. The war has resulted in thousands of deaths and displacement of millions of people. Both the Ukrainian government and Russian-backed separatists have been accused of human rights abuses and war crimes. Efforts to negotiate a peaceful resolution to the conflict have been unsuccessful.",
  "left_wing_commentary": "The left-wing view of the Ukraine war emphasizes the role of Russian aggression and the need for Ukrainian sovereignty. Left-wing commentators often criticize the Russian government for its interference in Ukrainian internal affairs and its support for separatist movements in Eastern Ukraine. They also argue for the provision of military aid to the Ukrainian government and support for international sanctions against Russia. Addition